In [1]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from ipywidgets import interact
import time

In [2]:
plt.rcParams['figure.dpi'] = 100
plt.rcParams['figure.figsize'] = (12,4)

# Optomechanical equations of motion

## Full nonlinear parametric coupling

The derivation here will follow the nice section in chapter 2 of the thesis of Frank Buters.

We will consider the case of optomechanics with the following dynamical variables:

* $x$ is the position of the mechanical oscillator
* $a$ is the complex field variable describing the cavity harmonic oscillator

The cavity is driven by an external input field $s_{in}$ via an external coupling rate $\kappa_{ext}$. The mechanics can be driven by external force $F_{ex}(t)$, which could, for example, represent a thermomechanical noise force. 

The equations of motion describing these dynamical variables are: 

\begin{eqnarray}
\frac{da}{dt} & = & \left[ i(\Delta - Gx(t)) -\frac{\kappa}{2} \right] a(t) + 
    \sqrt{\kappa_{ext}} s_{in}(t)\\
\frac{d^2x}{dt^2} & = & -\gamma_m \frac{dx}{dt} - \Omega_m^2 x(t) + 
    \frac{\hbar G}{m}|a(t)|^2 
\end{eqnarray}

Here, $\Delta = \omega_d - \omega_c$ is the detuning of the frequency $\omega_d$ of the external drive field with respect to the natural frequency $\omega_c$ of the cavity. $\gamma_m$ is the mechanical damping rate, $\kappa$ is the total cavity damping rate, and $\Omega_m$ is the natural frequency of the mechanical resoantor. The coupling constant $G$ (the "cavity pull parameter") describes the coupling between the two, and is given by the rate of change of the optical cavity natural frequency with respect to the position of the mechanical resonator:

$$
G = \frac{d \omega_c}{dx}
$$

Note that the in the above equations, we have gone into the "rotating frame" of the drive frequency. In the "lab frame", the field of the cavity $\alpha$ could oscillate at $\omega_c$ and $\omega_d$. Here, we remove the time dependence of the drive field by defining:

$$
\alpha(t) = a(t) e^{i\omega_dt}
$$

In this case, a drive field oscillating at a constant amplitude and phase is described by a constant, time-independent value of $s_{in}$ in these equations.
As usual when solving ODEs, we will split the second order equation for $x$ up into two first order equations: 

\begin{eqnarray}
\frac{da}{dt} & = & \left[ i(\Delta - Gx(t)) -\frac{\kappa}{2} \right] a(t) + 
    \sqrt{\kappa_{ext}} s_{in}(t)\\
\frac{dv}{dt} & = & -\gamma_m \frac{dx}{dt} - \Omega_m^2 x(t) + 
    \frac{\hbar G}{m}|a(t)|^2 \\
\frac{dx}{dt} & = & v
\end{eqnarray}

## Linearized equations

A common approximation is these linearize these equations, an approximation which is valid as long as the cavity is strongly driven: 

$$
s_{in}(t) = s_0
$$



If the cavity is strongly driven, we will first have a ring up to a large average steaday-statte amplitude $\bar a$. In this case we can define a small displacement of the cavity field about the strong, steady-state field amplitude:

$$
a(t) = \bar a + c(t)
$$

In addition, due to the radiation pressure force, the mechanical resonator will experience a static displacement $\bar x$. We will now define the position of the mechanical resonator with respect to its new steady-state average position: 

$$
x(t) = \bar x + u(t)
$$

Using the above equations, we can find the following for $\bar a$ and $\bar x$:

\begin{eqnarray}
\bar a & = & \frac{\sqrt{\kappa_{ext}} s_0}{-i(\Delta+\hbar G x) - \kappa/2} \\
\bar x & = & \frac{\hbar G}{m\Omega_m^2}|\bar a|^2
\end{eqnarray}

Using these with the full EOMs above, and discarding terms second order in $u(t)$ and $c(t)$, we obtain then the linearized EOMs of optomechanics:

\begin{eqnarray}
\frac{dc}{dt} & =  &  \left[ i \tilde \Delta - \frac{\kappa}{2} \right] + G \bar a u(t) \\
\frac{d^2u}{dt^2} & = & -\gamma_m \frac{du}{dt} - \Omega_m^2 u(t) + \frac{\hbar G \bar a}{m} [c(t) + c^*(t)] + \frac{F_{ex}(t)}{m}
\end{eqnarray}


where $\tilde \Delta = \Delta  + \hbar G x$. Similar to above, for numerical solutions, we split the second differential equation by defining $v = du / dt$ giving:

\begin{eqnarray}
\frac{dc}{dt} & =  &  \left[ i \tilde \Delta - \frac{\kappa}{2} \right] + G \bar a u(t) \\
\frac{dv}{dt} & = & -\gamma_m \frac{du}{dt} - \Omega_m^2 u(t) + 
    \frac{\hbar G \bar a}{m} [c(t) + c^*(t)] + \frac{F_{ex}(t)}{m}\\
\frac{du}{dt} & = & v
\end{eqnarray}
